# Superconductor Critical Temperature Prediction using DeepSet Neural Networks

**Research Paper Implementation**  
*Centre for Computational Natural Sciences and Bioinformatics*  
*International Institute of Information Technology Hyderabad*

---

## Overview

This notebook implements a DeepSet neural network for predicting superconductor critical temperature (Tc) based solely on chemical composition. The model:

- Trains 50 independent models with different random seeds
- Uses 22 elemental features from Mendeleev's periodic table + stoichiometry
- Achieves **R² = 0.9170** and **RMSE = 9.95 K** on ensemble-averaged predictions
- Provides SHAP-based interpretability analysis

## Dataset Requirements

**Place your dataset file in the `../data/` directory:**
- Expected filename: `superconductors_kaggle_ready_v2.csv`
- Columns: `element` (composition), `critical_temp_K` (target Tc)

---

In [1]:
# Option 1: Install from requirements file (recommended)
# !pip install -r ../requirements.txt

# Option 2: Install minimal packages directly
!pip install mendeleev shap mlflow "numpy>=1.26.0" "protobuf>=5.28.0"

Defaulting to user installation because normal site-packages is not writeable


## 1. Install Required Packages

Install necessary dependencies (run this cell first):

## 2. Import Libraries and Load Data

Import all required libraries and load the superconductor dataset:

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import mlflow
import mlflow.tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import re
import time
from mendeleev import element
import mendeleev
from collections import Counter
import matplotlib.pyplot as plt
import joblib
import os


print("LOADING AND PREPARING DATA")
# Load the dataset - Update path as needed for your environment
# For VS Code/local environment:
dataset_path = "../data/superconductors_kaggle_ready_v2.csv"

# For Kaggle (comment out the line above and uncomment below):
# dataset_path = "/kaggle/input/dataset1/superconductors_kaggle_ready_v2.csv"

# Check if file exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError(
        f"Dataset not found at {dataset_path}\n"
        f"Please place 'superconductors_kaggle_ready_v2.csv' in the ../data/ directory"
    )

df = pd.read_csv(dataset_path, sep=None, engine='python')

print(df.head())

# Rename columns to match the expected format
df = df.rename(columns={
    "element": "composition",
    "critical_temp_K": "Tc"
})

# Clean the target variable
df['Tc'] = pd.to_numeric(df['Tc'], errors='coerce')
df = df[df['Tc'].notna()]
y = df['Tc'].values

print(f"Final dataset shape after cleaning: {df.shape}")


print("PARSING CHEMICAL COMPOSITIONS")

# Extract chemical composition information
def parse_composition(composition_str):
    """Parse composition string to extract elements and their stoichiometry"""
    if pd.isna(composition_str):
        return {}

    # Finding all element number pairs
    pattern = r'([A-Z][a-z]?)(\d*\.?\d*)'
    matches = re.findall(pattern, composition_str)

    composition = {}
    for element, count in matches:
        count = float(count) if count else 1.0
        composition[element] = composition.get(element, 0) + count

    return composition


# Get mendeleev features for each element
def get_mendeleev_features(element_symbol):
    """Extract 22 features from Mendeleev periodic table"""
    try:
        el = mendeleev.element(element_symbol)

        features = {
            'atomic_number': el.atomic_number if hasattr(el, 'atomic_number') else np.nan,
            'atomic_volume': el.atomic_volume if hasattr(el, 'atomic_volume') else np.nan,
            'block': el.block if hasattr(el, 'block') else np.nan,
            'density': el.density if hasattr(el, 'density') else np.nan,
            'dipole_polarizability': el.dipole_polarizability if hasattr(el, 'dipole_polarizability') else np.nan,
            'electron_affinity': el.electron_affinity if hasattr(el, 'electron_affinity') else np.nan,
            'evaporation_heat': el.evaporation_heat if hasattr(el, 'evaporation_heat') else np.nan,
            'fusion_heat': el.fusion_heat if hasattr(el, 'fusion_heat') else np.nan,
            'group_id': el.group_id if hasattr(el, 'group_id') else np.nan,
            'lattice_constant': el.lattice_constant if hasattr(el, 'lattice_constant') else np.nan,
            'lattice_structure': el.lattice_structure if hasattr(el, 'lattice_structure') else np.nan,
            'melting_point': el.melting_point if hasattr(el, 'melting_point') else np.nan,
            'period': el.period if hasattr(el, 'period') else np.nan,
            'specific_heat': el.specific_heat if hasattr(el, 'specific_heat') else np.nan,
            'thermal_conductivity': el.thermal_conductivity if hasattr(el, 'thermal_conductivity') else np.nan,
            'vdw_radius': el.vdw_radius if hasattr(el, 'vdw_radius') else np.nan,
            'covalent_radius_pyykko': el.covalent_radius_pyykko if hasattr(el, 'covalent_radius_pyykko') else np.nan,
            'en_pauling': el.en_pauling if hasattr(el, 'en_pauling') else np.nan,
            'atomic_weight': el.atomic_weight if hasattr(el, 'atomic_weight') else np.nan,
            'atomic_radius_rahm': el.atomic_radius_rahm if hasattr(el, 'atomic_radius_rahm') else np.nan,
            'ionenergies': el.ionenergies.get(1, np.nan) if hasattr(el, 'ionenergies') and el.ionenergies else np.nan,
            'nvalence': el.nvalence() if hasattr(el, 'nvalence') else np.nan
        }

        return features
    except Exception as e:
        print(f"Error getting features for {element_symbol}: {e}")
        return {key: np.nan for key in ['atomic_number', 'atomic_volume', 'block', 'density',
                                        'dipole_polarizability', 'electron_affinity', 'evaporation_heat',
                                        'fusion_heat', 'group_id', 'lattice_constant', 'lattice_structure',
                                        'melting_point', 'period', 'specific_heat', 'thermal_conductivity',
                                        'vdw_radius', 'covalent_radius_pyykko', 'en_pauling', 'atomic_weight',
                                        'atomic_radius_rahm', 'ionenergies', 'nvalence']}


# Encode categorical features
def encode_categorical(value, categories):
    """One-hot encode categorical values"""
    encoding = [0.0] * len(categories)
    if pd.notna(value) and value in categories:
        encoding[categories.index(value)] = 1.0
    return encoding


# Define categories for encoding
blocks = ['s', 'p', 'd', 'f']
lattice_structures = ['BCC', 'BCT', 'FCC', 'HEX', 'ORC', 'RHL', 'TET', 'MCL', 'TRC']

print("EXTRACTING FEATURES")

# Process each composition
max_elements = 10  # Maximum number of elements to consider
feature_dim = 23  # 22 mendeleev features + 1 stoichiometry

X_deepset = []
valid_indices = []

for idx, comp_str in enumerate(df['composition']):
    composition_dict = parse_composition(comp_str)

    if not composition_dict:
        continue

    # Create feature matrix for this composition
    comp_features = []

    for element_symbol, stoich in composition_dict.items():
        mendeleev_feats = get_mendeleev_features(element_symbol)

        # Create feature vector for this element
        element_features = [
            mendeleev_feats['atomic_number'],
            mendeleev_feats['atomic_volume'],
            mendeleev_feats['density'],
            mendeleev_feats['dipole_polarizability'],
            mendeleev_feats['electron_affinity'],
            mendeleev_feats['evaporation_heat'],
            mendeleev_feats['fusion_heat'],
            mendeleev_feats['group_id'],
            mendeleev_feats['lattice_constant'],
            mendeleev_feats['melting_point'],
            mendeleev_feats['period'],
            mendeleev_feats['specific_heat'],
            mendeleev_feats['thermal_conductivity'],
            mendeleev_feats['vdw_radius'],
            mendeleev_feats['covalent_radius_pyykko'],
            mendeleev_feats['en_pauling'],
            mendeleev_feats['atomic_weight'],
            mendeleev_feats['atomic_radius_rahm'],
            mendeleev_feats['ionenergies'],
            mendeleev_feats['nvalence']
        ]

        # Add encoded categorical features
        element_features.extend(encode_categorical(mendeleev_feats['block'], blocks))
        element_features.extend(encode_categorical(mendeleev_feats['lattice_structure'], lattice_structures))

        # Add stoichiometry as the last feature
        element_features.append(stoich)

        comp_features.append(element_features)

    # Pad or truncate to max_elements
    if len(comp_features) > max_elements:
        comp_features = comp_features[:max_elements]
    else:
        # Pad with zeros
        while len(comp_features) < max_elements:
            comp_features.append([0.0] * feature_dim)

    X_deepset.append(comp_features)
    valid_indices.append(idx)

    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1}/{len(df)} compositions")

X_deepset = np.array(X_deepset)
y = y[valid_indices]

print(f"X_deepset shape: {X_deepset.shape}")
print(f"y shape: {y.shape}")


print("DEFINING DEEPSET MODEL")

def build_deepset_model(input_shape, phi_layers, rho_layers):
    """
    Build DeepSet model with separate phi and rho networks
    
    Args:
        input_shape: (max_elements, feature_dim)
        phi_layers: List of layer sizes for phi network
        rho_layers: List of layer sizes for rho network
    """
    inputs = layers.Input(shape=input_shape)

    # Phi network - applied to each element
    phi = inputs
    for units in phi_layers:
        phi = layers.TimeDistributed(layers.Dense(units, activation='relu'))(phi)

    # Permutation-invariant pooling
    pooled = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1))(phi)

    # Rho network - applied to pooled representation
    rho = pooled
    for units in rho_layers[:-1]:
        rho = layers.Dense(units, activation='relu')(rho)

    # Output layer
    output = layers.Dense(rho_layers[-1], activation='linear')(rho)

    model = Model(inputs=inputs, outputs=output)
    return model


# Define architecture
phi_layers = [992, 768, 512, 384, 256, 128, 300]
rho_layers = [960, 832, 768, 640, 512, 384, 256, 192, 160, 128, 96, 64, 1]

print(f"Phi network layers: {phi_layers}")
print(f"Rho network layers: {rho_layers}")


print("TRAINING MULTIPLE MODELS WITH DIFFERENT RANDOM SEEDS")

# Training parameters
num_runs = 50
batch_size = 64
epochs = 400
initial_lr = 0.001

# Create directories for saving models
os.makedirs("../saved_models", exist_ok=True)
os.makedirs("../results", exist_ok=True)

# Store results
all_results = []

# MLflow setup
mlflow.set_experiment("superconductor_deepset")

for run_idx in range(num_runs):
    print(f"\n{'='*80}")
    print(f"TRAINING RUN {run_idx + 1}/{num_runs}")
    print(f"{'='*80}")

    # Set random seeds for reproducibility within this run
    seed = 42 + run_idx
    np.random.seed(seed)
    tf.random.set_seed(seed)

    with mlflow.start_run(run_name=f"run_{run_idx + 1}"):
        # Log parameters
        mlflow.log_param("run_index", run_idx + 1)
        mlflow.log_param("random_seed", seed)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("initial_lr", initial_lr)

        # Split data with this run's random seed
        X_temp, X_test, y_temp, y_test = train_test_split(
            X_deepset, y, test_size=0.2, random_state=seed
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.175, random_state=seed  # 0.175 * 0.8 ≈ 0.14
        )

        print(f"Train set size: {len(X_train)}")
        print(f"Validation set size: {len(X_val)}")
        print(f"Test set size: {len(X_test)}")

        # Normalize features - create separate scaler for this run
        scaler = StandardScaler()
        X_train_flat = X_train.reshape(-1, feature_dim)
        scaler.fit(X_train_flat)

        X_train_norm = scaler.transform(X_train.reshape(-1, feature_dim)).reshape(X_train.shape)
        X_val_norm = scaler.transform(X_val.reshape(-1, feature_dim)).reshape(X_val.shape)
        X_test_norm = scaler.transform(X_test.reshape(-1, feature_dim)).reshape(X_test.shape)

        # Build model
        model = build_deepset_model(
            input_shape=(max_elements, feature_dim),
            phi_layers=phi_layers,
            rho_layers=rho_layers
        )

        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
            loss='mse',
            metrics=['mae']
        )

        # Callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=40,
            restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=15,
            min_lr=1e-7
        )

        # Train model
        history = model.fit(
            X_train_norm, y_train,
            validation_data=(X_val_norm, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        # Evaluate on test set
        y_pred_test = model.predict(X_test_norm).flatten()

        test_r2 = r2_score(y_test, y_pred_test)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        print(f"\nTest Set Performance:")
        print(f"R² Score: {test_r2:.4f}")
        print(f"MAE: {test_mae:.4f} K")
        print(f"RMSE: {test_rmse:.4f} K")

        # Log metrics
        mlflow.log_metric("test_r2", test_r2)
        mlflow.log_metric("test_mae", test_mae)
        mlflow.log_metric("test_rmse", test_rmse)

        # Save model and scaler
        model_path = f"../saved_models/model_run_{run_idx + 1}.keras"
        scaler_path = f"../saved_models/scaler_run_{run_idx + 1}.pkl"

        model.save(model_path)
        joblib.dump(scaler, scaler_path)

        mlflow.log_artifact(model_path)
        mlflow.log_artifact(scaler_path)

        # Store results
        all_results.append({
            'run': run_idx + 1,
            'seed': seed,
            'test_r2': test_r2,
            'test_mae': test_mae,
            'test_rmse': test_rmse,
            'train_size': len(X_train),
            'val_size': len(X_val),
            'test_size': len(X_test)
        })

# Save all results
results_df = pd.DataFrame(all_results)
results_df.to_csv("../results/training_results.csv", index=False)

print("\n" + "="*80)
print("TRAINING COMPLETED FOR ALL RUNS")
print("="*80)
print(f"\nSummary Statistics Across {num_runs} Runs:")
print(f"Average Test R²: {results_df['test_r2'].mean():.4f} ± {results_df['test_r2'].std():.4f}")
print(f"Average Test MAE: {results_df['test_mae'].mean():.4f} ± {results_df['test_mae'].std():.4f} K")
print(f"Average Test RMSE: {results_df['test_rmse'].mean():.4f} ± {results_df['test_rmse'].std():.4f} K")
print(f"\nResults saved to: ../results/training_results.csv")
print(f"Models saved to: ../saved_models/")


2026-02-08 12:14:36.908543: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


LOADING AND PREPARING DATA
                            comment  curiet  debyet  \
0  G-L = 1.07. GAPENE from SP-heat.     NaN   418.0   
1                               NaN     NaN     NaN   
2        OZ=3.99-3.              97     NaN     NaN   
3            Hc2(0) shows Hc2(4.2K)     NaN     NaN   
4                               NaN     NaN     NaN   

                      element  gapene  kaizero_value  moment_value  \
0                        C2Y1    3.64            NaN           NaN   
1  Nd1Fe0.5Ru0.5As1F0.11O0.89     NaN            NaN           NaN   
2        Nd1.85Th0.15Cu1O3.98     NaN            NaN           NaN   
3                Nb70Mo10Si20     NaN            NaN           NaN   
4               Tl2Ba2Cu1O6+Z     NaN            NaN           NaN   

              name  neelt  nhc2zero_value  ...  resistivity_roomtemp_uOhmcm  \
0              YC2    NaN             NaN  ...                          NaN   
1      Nd(O,F)FeAs    NaN             NaN  ...                 

/home/Tj/.local/lib/python3.13/site-packages/mendeleev/models.py:397: UserWarning: Se has multiple allotropes, check <{self.symbol}.phase_transitions> for details.
  warnings.warn(


Error getting features for Z: Element not found: Z
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz


/home/Tj/.local/lib/python3.13/site-packages/mendeleev/models.py:397: UserWarning: S has multiple allotropes, check <{self.symbol}.phase_transitions> for details.
  warnings.warn(


...

Error getting features for Z: Element not found: Z
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz
Error getting features for X: Element not found: X
Error getting features for X: Element not found: X
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z


/home/Tj/.local/lib/python3.13/site-packages/mendeleev/models.py:397: UserWarning: Sn has multiple allotropes, check <{self.symbol}.phase_transitions> for details.
  warnings.warn(


Error getting features for Z: Element not found: Z
Error getting features for M: Element not found: M
Error getting features for M: Element not found: M
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for X: Element not found: X
Error getting features for X: Element not found: X
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for X: Element not found: X
Error getting features for X: Element not found: X
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Oz: Element not found: Oz
Error getting features for Oz: Element not found: Oz
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for 

Exception ignored in: <function _collection_gced at 0x7f0d46ed67a0>
Traceback (most recent call last):
  File "/home/Tj/.local/lib/python3.13/site-packages/sqlalchemy/event/registry.py", line 103, in _collection_gced
    def _collection_gced(ref: weakref.ref[Any]) -> None:
KeyboardInterrupt: 


Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z
Error getting features for Z: Element not found: Z


## 3. Ensemble Predictions on Test Compositions

Make predictions using the ensemble of 50 trained models on the Hosono dataset compositions:

In [ ]:
new_compositions = [
    "La3Ni2O7",
    "B4Mg",
    "Mo0.94Tc1.06",
    "C4Mo2.18Nb1.82",
    "C120Rb8",
    "C24KO2V",
    "Ba0.6BiK0.4O2.3",
    "B3Mg3N",
    "C70CsK2",
    "ClHfNNa0.125",
    "Nb2.802SnZr0.198",
    "Mo6S8Sr",
    "Nb2.919SnV0.081",
    "Nb3Sn0.81Ti0.19",
    "C0.05N0.77Nb0.83Ti0.17",
    "Ga1.04Nb6Sn0.96",
    "Li0.88N3.48Nb3O0.52",
    "Ga3Ru",
    "Ag0.961Mo6S8",
    "Nb2.61Sn0.9Ta0.39",
    "B0.24O0.36V3Zr3",
    "Mo3Pt",
    "Ga0.1NNb0.9",
    "N2NbTi",
    "Bi2Ca",
    "ClHfNNa0.288",
    "HNb3Sn",
    "B5Pd14Y2",
    "La0.0039SiV2.9961",
    "GaNb5Sn2",
    "Ba0.65Bi0.75K0.31La0.29O3",
    "Sb0.4Si1.6V6",
    "Ga40.392Mo7.396",
    "C6.156B1.764La5",
    "B12Sc0.4Zr0.6",
    "GaNi0.15V2.85",
    "BrHfN",
    "C59Na3.82",
    "Si2V5.442Zr0.558",
    "C16Br4P2Ta",
    "C60Ba2.9K3.07",
    "C60Rb",
    "Mn3Mo18S24",
    "N4Nb2.8Ti1.2",
    "LiO2Ti",
    "BBe2",
    "BrNZr",
    "Li0.64NbO2",
    "Al2CaGe2",
    "Al0.48B2.2Mg0.52",
    "CoGa5U",
    "GaSiV6",
    "B4.026Mo2",
    "B5Ir2.5Mo2.5",
    "CB2Ni2Tb0.082Y0.918",
    "Na0.11O3W",
    "NbZr",
    "Mo0.2Re0.8",
    "Ga0.15Nb0.85",
    "B6.6Ir0.7Mg2",
    "BLuPd3",
    "GaNb1.8V1.2",
    "Al0.45B2Mg0.56",
    "HfTaV4",
    "Al5.598GeW1.402",
    "Ba0.69Bi0.66Gd0.35K0.3O3",
    "HfV4Zr",
    "Au0.5Nb3Pt0.5",
    "Hf0.5Nb0.2V2Zr0.3",
    "Mo1.45Re0.55",
    "N0.77Nb",
    "B7Mo3Y",
    "Hf0.4Nb1.2Zr0.4",
    "Au0.3Nb3Pt0.7",
    "Ag1.91Cs1.16Mo9S11",
    "C3B4Ni4Y3",
    "N0.86Nb2O0.14",
    "Nb0.666Ti0.666Zr0.666",
    "ClHfNNa0.25",
    "Nb3Tl",
    "LiMoS2",
    "NbTi",
    "CNb1.91",
    "GaTi0.201V2.799",
    "Bi2.9375Nb",
    "C19Yb15",
    "Si2Ta0.8V5.2",
    "C19Er10Ru10",
    "B2La3N2.91Ni2",
    "GeSiV6",
    "ErNb6Sn5",
    "Mo6S8Sn0.854",
    "C36RhZn9",
    "TcW",
    "Mo6OsRu",
    "InLa3N",
    "B32Rh27.2Ru4.8Y8",
    "Mo0.81Ru0.19",
    "HfTc",
    "Tc0.85W0.15",
    "HfNb",
    "Au0.5Nb1.5",
    "Mo1.3Rh0.7",
    "La2.667Se4",
    "INTi",
    "P2Pb2",
    "B2BaRh2",
    "MoTc",
    "Nb0.3Ti0.7",
    "Mn0.51SiV2.49",
    "Ga2Nb3",
    "Hf3N4",
    "BaSi6",
    "BRh3SiY",
    "B0.1V1.9Zr",
    "Nb14S5",
    "Sn0.85Zn0.15",
    "Au0.3Ge0.7Nb3",
    "CTl",
    "B1.76Be0.24Zr",
    "Co2Mo3",
    "Ga0.375V0.625",
    "Cd0.05Pb0.95",
    "Ga40.88Mo8S0.86",
    "Au0.938Cr0.062Nb3",
    "Ir0.88Mo2.12",
    "O0.67V3Zr3",
    "Al0.45Mg1.55Si",
    "N0.43Ta",
    "In1.39Mo15Sc1.91Se19",
    "Au0.3Nb0.7",
    "B4Nb3",
    "CoNb4P",
    "Mo9Se11",
    "Pb4.998S1.002",
    "Mo0.09Nb2.91Sn",
    "Sn0.6Tl0.4",
    "B2SiV5",
    "Bi3Se4",
    "Si1.2Sn0.8V6",
    "B2Ta0.5Zr0.5",
    "GaNb6Pt",
    "Si2Ti1.122V4.878",
    "Pb5",
    "P2Pt6Sr",
    "Nb0.2Ti0.8",
    "Pb2.4Tl1.6",
    "N9Nb3.72Ta4.28",
    "MgNi2",
    "Cd3Mo18S24",
    "Ir0.94Mo3.06",
    "HfTa",
    "Bi1.185K0.57O3.4",
    "Pb0.64Sb0.36",
    "CB2Ni2Tb0.328Y0.672",
    "Nb14.5Re43.5",
    "Au0.75Ir0.25Nb3",
    "H0.07NbSe2",
    "Mo18Se24Zn3",
    "Nb6Sb1.6Sn0.4",
    "Hf1.2Nb1.8Sn",
    "BaBiSeSn",
    "NbTa",
    "Pb3.4Tl0.6",
    "C4Th4",
    "ReW",
    "B4Y",
    "RhSe",
    "C2BBr2La3",
    "HfTi",
    "Mo1.2Si2V4.8",
    "B2Mo0.72Nb0.28",
    "In2.4Pb1.6",
    "Mo0.3Tc1.7",
    "Nb6RhRu",
    "CAl0.1W0.9",
    "B24Mo10U5",
    "Pb1.2Sn2.8",
    "C2B3Y2",
    "C5Nb6",
    "Rh3Se8",
    "Hg3",
    "B2MgNi2.5",
    "BN0.975Nb2",
    "I1.76Mo6S5.96",
    "Ba3.04Bi3.37K0.96Na0.63O12",
    "RuV",
    "Cr1.2GaV1.8",
    "Ge0.125Pb0.875",
    "Nb3Sn2Ti3",
    "B1.1Rh",
    "B0.6N0.4Zr",
    "Pt7Sc4Si2",
    "Ru0.6W0.4",
    "Nb0.33Ta0.33V0.34",
    "Cs0.22O3W",
    "Li0.795NbO2",
    "Mo9Se11Tl2",
    "Rh5Ti3",
    "C70Cs1.09Na2.24",
    "C2HoNi",
    "ReTc",
    "Nb6PdRh",
    "SiSnV6",
    "NbV",
    "C2Ni0.96Th2",
    "Nb0.33U0.33Zr0.34",
    "B6Ta5",
    "Sb0.32Sn3.68",
    "B2Mo0.5Zr0.5",
    "Ga0.51Sn0.49V3",
    "N4Zr3",
    "Re25Zr21",
    "EuMo6S8",
    "NbRhSi",
    "N0.58Nb",
    "PZr",
    "Ga1.02Sb0.98V6",
    "Re5.68V2.32",
    "Au6Ba8Si40",
    "Pb0.26Sn0.74",
    "Pd0.08Sn0.92",
    "MoNTa",
    "Re22.8V7.2",
    "C0.96Mo1.93",
    "Ge3V5",
    "Ga1.5IrNb5",
    "NNbO",
    "Mo0.8Ru1.2",
    "Ge1.2Sn0.8V6",
    "CY2",
    "Ba7.96Ga10Si34.01",
    "TaTi",
    "Rh3S3",
    "Lu2Ru3Si5",
    "BePd",
    "Ba7.87Ge11.97Si33.31",
    "Re4Si2Th",
    "Ba0.7BiK0.3O2.91",
    "C2Ce0.5Lu0.5",
    "FLuS",
    "Ba2Si4",
    "Ti0.05V0.95",
    "NbU",
    "Mo4N7",
    "AuSb3",
    "Mo1.95Os1.05",
    "In1.74Sn0.26",
    "SbTl",
    "CClLa2",
    "Ba4Bi0.4O12Pb3.6",
    "AlBMo",
    "Ir3Zr",
    "C4.79Ni4.96Th3",
    "MoRe2",
    "Ge1.305Pt0.67Y",
    "La9.6S16Y2.4",
    "ClN2NaSZr2",
    "BGe0.3Mo1.7",
    "Co0.3GaV2.7",
    "Ti0.89V0.11",
    "CAu",
    "C2Br3La3",
    "C2Th",
    "Ge22Mo11.75Rh1.25",
    "TeTl2",
    "Re0.975V0.025",
    "B2Ta0.5Ti0.5",
    "N0.81V",
    "Bi4O7Rb2",
    "Si3Yb",
    "IrTa",
    "Ge2W",
    "B3Nb3Ru3",
    "CB2ErRh2",
    "B4Ru5Sc2",
    "N2.62Nb4",
    "BiKO2",
    "RhScSi2",
    "Ca0.08CuLa1.92O3.99",
    "BaCu2O7SrTlY",
    "Ba2Cu2.92O6Y",
    "As2Ba0.52Fe2K0.48",
    "Br2CuO2Sr2",
    "Ba2Ca0.14Co0.12Cu2.88O6.96Y0.86",
    "Ba1.9Ca0.2Cu3La0.1O7Y0.8",
    "Ba4Cu6.91O14.94Y2",
    "CuLa2O4.178",
    "As2Ba0.5Fe2K0.4Sn0.1",
    "Ba2Ca0.9Cu2.14O8Tl1.96",
    "Ba2Cu2.89O6.66Y",
    "Ba2Cu2.88LaO6.4",
    "BaCa0.2Cu2.8O7.8P0.2SrY0.8",
    "Ba11Bi9Cu4O29",
    "CuNd2O3.864",
    "As10Ca6Fe7.627Pt6.373",
    "Cu12.0032Na21O50",
    "BaCuFeO5Y",
    "Ba4Cu6.72Nd2O15.08",
    "As2Ba0.61Fe2K0.35Sn0.04",
    "Ba4Cu6.74O14.6Y2",
    "Ba2Bi0.22CuHg0.78O4.28",
    "Ba1.93Cu3K0.07O7Y",
    "Fe15.631K8.622Se20",
    "Ba2Cu2.79Fe0.15O6.96Y",
    "As3CaFe4",
    "Ba2.668Ce1.332Cu5.88O17.8Sm4",
    "As2Ba0.32Fe2K0.68",
    "CrCu4O19.9Sr8Tl3",
    "Ca0.5Cu2O6.3Pb0.46Sr2.46Y0.5",
    "Ba2Cu3O6.072Y",
    "Ba4Cu6.98O14.8Y2",
    "Ba2Co0.03Cu2.97O6.94Y",
    "Ba2Cu3.904Fe0.052O7.895Pr0.19Y0.79",
    "As2Ba0.68Fe2K0.24Sn0.07",
    "Ca0.12CuLa1.88O3.99",
    "Bi3.86Ca0.87Cu2.74O14Sr3.62",
    "Fe15.29K9.223Se20",
    "Ca1.1Cu2La1.9O5.95",
    "Ba2Cu3La0.7O6.97Pr0.3",
    "C0.144Ba2Cu2.85O7.162Y",
    "Ba1.95Cu3Nd1.05O6.95",
    "Ba4Cu6.76O14.54Y2",
    "Ba1.2Ca0.6Cu3La0.9O7.11Y0.3",
    "Au0.17Ba2Ca2Cu3Hg0.69O8.3",
    "Ag1.592Cu0.4Pd3Se4",
    "As2Ba0.34Fe2K0.66",
    "Ca0.1CuLa1.9O4.058",
    "CuLa1.85O3.6Sr0.15",
    "Cu2GdLa0.098O8RuSr1.902",
    "Ba1.98Cu3O7Pr0.11Y0.91",
    "CuLa1.94Na0.06O4",
    "Cu2.97Mo18S24",
    "Co0.015Cu0.985La1.8O4Sr0.2",
    "As2Ca0.73FePr0.27",
    "Ba4Cu6.72O14.63Y2",
    "Ba3.87Ca0.246Cu6.73O14.63Y1.884",
    "Bi0.45CaCu2.55O7.07Sr2",
    "Al0.42Ba4Cu6.58Er2O14.88",
    "Bi2.1Ca2Cu3O10Sr1.9",
    "Ce0.3CuNd1.7O4",
    "Ba2Cu2.5Fe0.5O7Y",
    "CaCu2.5LaO6.952Pb0.5Sr",
    "Ba1.6Ca0.1Cu3O6.104Sr0.4Yb0.9",
    "Cu1.71O3Sr0.57Y0.43",
    "Ba1.24Ca0.51Cu3La0.99Nd0.26O7.085",
    "CuLa1.978O4Sr0.022",
    "Ba4Cu6.84Ho2O14.36",
    "Cu3O6.84Sr2Y",
    "Bi0.33Ca0.89Cu2O7Sr2Tl0.78",
    "Ca0.81Cu2O7Sr2Tl1.19",
    "Co0.025Cu0.975La1.8O4Sr0.2",
    "Ba1.98Cu2.91O6.47Y0.98",
    "CCa1.11Cu2O7Sr1.89",
    "Bi10Cu5O29Sr10",
    "Ba1.32Ca0.344Cu3La1.13O7.18Y0.172",
    "Ba2Ca0.424Cu2O7Pr0.576Tl",
    "Al0.16Ba2Ca0.05Cu2.84O6.75Y0.95",
    "BaBiCu",
    "Ba4Cu6.83O14.6Y2",
    "Ba2Cu2NbNdO7.86",
    "Ba2Cu3O7.34Y",
    "Ba1.4Cu3O6.94Sr0.6Yb",
    "BaCu2.9ErO7.01P0.1Sr",
    "Ba2Cu3NdO6.92",
    "Cu1.98Fe1.02O7.36Sr2Y",
    "Ba2Ce0.5Cu2Eu1.5O9Tl",
    "Ce0.165CuNd1.835O3.8",
    "Bi2CaCu2O9.07Sr2",
    "Cu0.15O6Ru0.85Sr2Y",
    "Cu1.5LaO5.008Pb0.5Sr",
    "Ba1.5Cu3Nd1.5O6.96",
    "Ba1.96Ca0.93Cu1.96O8.17Tl2",
    "CuLa2O3.95",
    "Ba2Ca0.29Cu3Er0.71O6.74",
    "Ba2Cu3LaO6.7",
    "Ba2Cu3O6.958Y",
    "Ba2Ca0.18Co0.24Cu2.76O7Y0.82",
    "Cu0.73LaMo0.25O3Zn0.03",
    "Fe4Si9.9Y1.2",
    "BaCuO5Tm2",
    "Ba1.6Cu3O6.49Sm1.4",
    "CCa2CuO5",
    "Ba2Ca0.83Cu2O6.75Tl1.17",
    "Ce0.05CuNd1.95O4",
    "Ba2Cu3NdO6.52",
    "Ba2Cu2.88O6.61Y",
    "BaCa0.49Cu2.8La1.51O6.48Pt0.2",
    "B8Fe3Nb7",
    "Ba1.5Ca0.5Cu3LaO6.72",
    "Cu2Gd2O9Pb0.5Sr2Tl0.5",
    "Ce0.58Cu2Eu1.8O10.024Ru0.92Sr1.7",
    "Cu2GdO7.96RuSr2",
    "Ba2Cu3O6.96Yb",
    "Ba1.2Ca0.6Cu2.977Dy0.3La0.9O7.004",
    "Ba2Ca2Cu3.18Hg0.82O12.13",
    "CuEu0.02O1.96Sr0.98",
    "Ca0.1CuLa1.9O3.99",
    "Ba1.04Ca0.78Cu2O7Sr0.96Tl1.22",
    "Ba3.888Ca0.17Cu6.73O14.4Y1.942",
    "Bi0.3Ca1.74Cu3Hg0.7O8.91Sr2.26",
    "Ba2Ca0.93Cu2K1.12O8.46Tl0.88",
    "Fe0.925Se",
    "Ba2Cu2Hg1.75O8.02Pb0.25Y",
    "Bi2.08CuO6Sr1.84",
    "CuNd1.2O3.765Sr0.4Y0.4",
    "Br2Ca2CuO2",
    "Ca0.08CuLa1.92O4.07",
    "Ba1.75Cu3La0.25O7Y",
    "Ba1.82Cu3K0.18O7Y",
    "Ba2CuHgO4.069",
    "Ca1.85Cu3.22O10Sr2Tl1.93",
    "Ba2Ca2Cu3Hg0.692O8.6",
    "Cu1.83Mo3S4",
    "Ba2Ce0.15CuHg0.85O4.15",
    "Ca0.2Cr0.15Cu2.85O8Pb1.75Sr2Y",
    "BaCu3O6.48SrY",
    "Ca0.12CuLa1.88O3.914",
    "Ba4Cu6.92O14.33Y2",
    "As2Ba0.78Fe2K0.12Sn0.1",
    "Ca0.27Cu2.969O7.73Pb2Sr1.968Y0.73",
    "Ba2Cu2.93O7Y",
    "Ba2Ca1.856Cu3.276O10Tl1.864",
    "Ba2Cu2.84O6.58Y",
    "CuO5Pr0.6Sr1.6Tl0.8",
    "Cu2GdO7.94RuSr2",
    "Ba8Ca8Cu12O39Tl7",
    "Ba2CuHgO4.054",
    "Ba2Cu2.94LaO7",
    "Co0.1Cu0.9La1.8O4Sr0.2",
    "Ba1.33Ca0.18Cu3La1.14O7.32Pr0.14",
    "Al0.1Ba2Cu2.89O6.14Y",
    "Ba1.9Cu3EuO6.93Pr0.1",
    "CuGd0.89La0.9O4Sr0.21",
    "Al0.11Ba2Cu2.89HoO6.53",
    "Ba0.72Ca1.84Cu3O9Sr1.28Tl1.16",
    "Ca0.73Cu2O4Sr1.19",
    "Ba1.936Cu3O7Pr0.532Y0.532",
    "Ba4Cu3DyO9.09",
    "Ba1.6Cu3O6.999Sr0.4Yb",
    "Ca0.12CuLa1.88O4.058",
    "Ba1.8Cu3La0.2O7.05Y0.94",
    "Ba2Co0.391Cu2.609O7.23Y",
    "Ba2Cu2.98O6.96Y",
    "Ca1.04Cu2.5Hg0.7O7.5Sr1.96Tl0.8",
    "Bi0.5Ca0.4Cu2Hg0.5Nd0.6O6.6Sr2",
    "Al0.11Ba2Cu2.725O6.4Tb0.182Y0.8",
    "Ca0.91Cu2O7Pb0.5Sr2Tl0.59",
    "BaFe1.95Se3",
    "Al0.06Ba2Ca0.06Cu2.94O6.81Y0.94",
    "Ba1.7Cu3Nd1.3O6.8",
    "Ba2Cu1.16Hg0.84O4.19",
    "Ba2Cu2.55Fe0.45O7.2Y",
    "Bi0.5CaCu1.5O4Sr",
    "Bi1.8Ca1.2Cu2.2O8.22Sr1.8",
    "Ba1.8Cu3EuO6.94Pr0.2",
    "Ba2Ca0.1Cu3O7Pr0.1Y0.8",
    "Ba1.5Ca2Cu3O9.784Sr0.5Tl1.81",
    "Ba1.5Ca0.5Cu3LaO6.98",
    "C0.5Cr0.15Cu2N0.5O10Sr4Tl0.85",
    "Ba2Cu1.059O6Tl1.941",
    "CCu1.85Fe0.3O10Sr4Tl0.85",
    "Ba1.9Cu3Eu1.1O7",
    "As6Ca2.56Fe7.49Na0.44Nb0.51",
    "Ba1.92Ca1.9Cu2.91O10Tl2.27",
    "CuLa1.908O3.66Sr0.092",
    "Au0.16Ba4Cu6.84Er2O15",
    "Ba2Cu2.64Fe0.36O6.41Y",
    "Al0.58Ba1.47Ca0.53Cu2.42LaO6.68",
    "Ba2Cu3O5.72Y",
    "As2Ba0.9Fe2Sn0.1",
    "Cu1.2Pd1.8Y",
    "Ba2Ca0.93Cu2O7.86Tl1.81",
    "Bi2.15Ca0.75Cu2O8Sr1.92",
    "BaCu3NdO7.13Sr",
    "Cu4.23Mo18S24",
    "Ba2Ca0.07Cu3O6.85Y0.93",
    "Ba2Cu2.85O7Re0.15Y",
    "BaCa0.4Cu2.8O7.8P0.2SrY0.6",
    "CuLa0.7O5SrTl1.3",
    "Ba1.8Cu3O6.97Sr0.2Yb",
    "Ba1.5Ca0.5Cu3LaO7.06",
    "Ba1.5Ca0.5Cu3LaO6.64",
    "Au0.099Ba2Cu2.901O4.5Y",
    "Ba2Cu2O10Y4",
    "CaCu2O3",
    "Ba1.986Ca0.035Cu4O8Y0.979",
    "Al0.1Ba2Cu2.874O6.31Y",
    "Ce0.4Cu2Gd1.6O9Pb0.5Sr2Tl0.5",
    "Ba2Cu2.9O7.049Re0.1Y",
    "Ba2Cu3NdO6.57",
    "BaCuO6SrTl2",
    "CuO4Sm1.9Sr0.1",
    "Ba2Ca0.09Cu3Er0.91O6.92",
    "BaCa0.25Cu2.8O7.8P0.2SrY0.75",
    "Au0.099Ba2Cu2.901O6.5Y",
    "Ba2CuHg0.96O4.34",
    "Ba2Ca2Cu3O8.84Tl0.93",
    "Ca0.48Cu2O6.96Pr0.52Sr2Tl0.94",
    "Bi2Cu2O8.5PrSr2",
    "Ce0.7Cu2Gd1.3O9.74RuSr2",
    "Ba2Cu2.85O6.94YZn0.15",
    "CuO2.02Pr0.17Sr0.83",
    "Ba1.6Cu3O6.201Sr0.4Yb",
    "CuLaNdO4",
    "CuLa1.85O3.96Sr0.15",
    "Ba2Cu2.88LaO6.8",
    "Cu2O7.76RuSr2Y",
    "Ce0.7Cu2Gd1.3O9.8RuSr2",
    "Ba1.5Ca0.5Cu3LaO6.32",
    "Ba2Cu2.94Fe0.06O6.36Y",
    "Ba4Cu6.78Nd2O14.85",
    "Ba25.96Cu64Eu6.04P120",
    "Ba1.936Cu3O7Pr0.432Y0.632",
    "Al0.3Ba1.92Ca0.2Cu2.7O6.76Y0.88",
    "Ce0.27Cr0.35Cu2.65O9Sr2Y1.73",
    "Ca0.91CuO2Sr0.09",
    "Ba2Cu2.88Fe0.12O6.26Y",
    "Ba1.6Ca0.35Cu3O6.075Sr0.4Yb0.65",
    "Cu3NbO8Ta",
    "BiCu2O8PbSr2Y",
    "CCr0.54Cu2Hg0.46O9.88Sr4",
    "Cu2FeO7.12Sr2Y",
    "Ba2Cu2.97O6.89Tm",
    "Bi0.3Ca3Cu4Hg0.7O10.74Sr2",
    "Ba2Ca0.15Co0.36Cu2.64O7.05Y0.85",
    "Fe2ScSi2",
    "Ba1.932Cu3O7Pr0.634Y0.434",
    "Ba2Cu5F14",
    "Bi2CaCu2O9Sr2",
    "B0.6Cu2.4O6.13Sr2Y",
    "Ba1.9Ca0.94Cu1.9O8Tl2.26",
    "Cu1.999ErFe1.001O7.36Sr2",
    "Ba2Cu2.5O7Pd0.5Y",
    "Bi2Ca0.3Cu4O10Sr2.7",
    "Au0.08Ba2Cu2.92O7.04Y",
    "Ba2Ca0.4Cu2Hg0.73O6.6Y0.6",
    "Ba4Cu7O14.3Y2",
    "Cu2GdIrO8Sr2",
    "Ba2Cu2.77Fe0.23O7.13Y",
    "Ba1.94Cu3O7Sm",
    "Ba2Cu2.79Fe0.15O6.92Y",
    "Ca6.925Cu23.15Nd3.66O41Sr2.925Y1.34",
    "Ba1.73Cu3Nd1.27O6.86",
    "Ba4Cu6O13Y2",
    "Ba2Cu3ErO6.45",
    "Cu2.601Mo9S12",
    "C0.35Ba2Cu2.95O7.65Y",
    "Bi2.09CaCu2O8.22Sr1.9",
    "Ba2Ca1.93Cu2.862O9Tl1.07",
    "Ba4Cu3GdO9.18",
    "Fe0.056SbTi2.944",
    "Ba4Cu6.77O14.26Y2",
    "Ba2Co0.17Cu2.83O6.54Y",
    "Ba2Cu3NaO6",
    "Ba1.5Ca0.5Cu3LaO6.18",
    "Bi1.916CuO5.482Sr1.84",
    "Ba2Ca0.72Cu2O8Tl2.16",
    "Ba2Cu3O6.967Y",
    "Ba2CaCu2.13Hg0.87O6.64",
    "Ba1.12Ca0.57Cu3La0.83O7.07Pr0.27",
    "Cu2.04Gd0.88O8.02Ru0.96Sr2.12",
    "Ce0.5Cu2Eu1.5Hg0.75O9Sr2W0.25",
    "AuBa2Ca0.3Cu2O7Y0.7",
    "CuLa1.58O4Pb0.27Sr0.15",
    "Ba2Cu3LuO6",
    "BaCuLu2O5",
    "Ba2Cu3ErO6.98",
    "CBi1.5Cu2O11Pb0.5Sr3.5",
    "Ba1.6Ca0.2Cu3O6.166Sr0.4Yb0.8",
    "Ba2Cu3O6.877Y",
    "Ba1.968Cu3O7Pr0.316Y0.716",
    "Ba1.95Cu3Na0.05O7Y",
    "Ba4Ca5Cu7Hg1.44O20Re0.5",
    "Ba2Ca1.9Cu3O10.94Tl1.82",
    "Ba2Cu2.91O6.58Y",
    "Ba0.15CaCu2La1.85O6",
    "Fe4Sc1.2Si9.86",
    "Ba2Ca0.84Cu2O8Tl1.94",
    "CuLa1.76O3.92Sr0.24",
    "Cu2.45Gd1.74Mo0.55O8Sr1.26",
    "CuLa2O4.024",
    "Ba2Cu3ErO6.99",
    "Ba4Cu6.87Er2O15",
    "Ba2Ca0.2Cu2Hg1.4O8Tl0.6Y0.8",
    "Ba2Ca0.27Co0.36Cu2.64O7.05Y0.73",
    "CaFe2P2",
    "Ba1.8Cu3Eu1.2O6.67",
    "Ba4Cu6.9Er2O14.3",
    "Fe1.13S0.05Te0.95",
    "Ba1.71Cu3K0.29O7Y",
    "BaCu3O7.03SrY",
    "Ba2Cu2.941O6.822Y",
    "Ca0.12CuLa1.88O4.05",
    "Ca0.77Cu2Hg0.3Nd0.23O7Pb0.7Sr2",
    "Cu2O5S2Sc2Sr3",
    "Ba1.5Ca0.5Cu3LaO6.9",
    "Ba2Ca1.07Cu2O8Tl1.93",
    "CuF1.08La0.813O1.92Sr0.187",
    "Cu2.1Mo6S4.5Se3.5",
    "Ba1.96Cu3K0.04O7Y",
    "Ba2Ca0.15Cu3O6Y0.85",
    "Ba1.988Ca0.043Cu4O8Y0.969",
    "Cu2ErO7.84RuSr2",
    "Ba2Cu4O8.06Y",
    "Cu1.999EuFe1.001O7.36Sr2",
    "CaCu2O7Pb0.7Sr1.82Tl0.3",
    "Ba1.4Ca0.172Cu3La1.31O7.16Y0.086",
    "Ba2CuHgO4.172",
    "Ba1.6Ca0.25Cu3O6.072Sr0.4Yb0.75",
    "Ba2CaCu2Hg0.7O7.8Tl1.3",
    "Ba2Ca0.5Cu2Nd0.5O6.86Tl0.95",
    "CBa4Ca0.7Cu5O14Y1.3",
    "Ba2Ca0.15Cu3Er0.85O6.79",
    "Bi0.5Ce0.85Cu2O8Sr1.5Y1.15",
    "Ba1.43Ca0.11Cu3La1.33O7.06Pr0.03",
    "Ba2Cu3NdO6.77",
    "Ba1.8Ca0.19Cu3La0.2O7.08Y0.81",
    "Al7.02Fe4.98Y",
    "CuHg0.75Mo0.25O4.6Sr2",
    "BaCaCu3LaO7.06",
    "Ca0.25Cu2.24Er1.11O7Pb0.76Sr1.64",
    "Ba1.6Cu3O6.35Sr0.4Yb",
    "Ba2Ca0.15Co0.12Cu2.88O7.01Y0.85",
    "Ba2CuHgO4.24",
    "CuLa0.08O1.89Sr0.92",
    "Ba2CuHg0.98O4.34",
    "Ce0.27CuNd1.32O3.93Sr0.41",
    "As6Ca3Fe7.038Pd1.962",
    "CaCu2La2O6.037",
    "Cu0.75Ni0.25O2Sr",
    "Ba2CaCu2O8Tl1.81",
    "BaCu2NdO7SrTl",
    "CBaCuO5.05Sr",
    "Ca0.1CuLa1.9O4.05",
    "Ba1.7Ca0.2Cu3La0.3O7Y0.8",
    "Cu2.2Ni0.8O8Pb2Sr2Y",
    "Ba2Co0.084Cu2.916O7.01Y",
    "Fe4GdP12",
    "Ba2Cu2.7O6.75YZn0.3",
    "Ba2Cu3NdO6.9",
    "BaCa0.432Cu2O6.98Pr0.568SrTl0.964",
    "Ba1.9Ca1.9Cu3O9Tl1.1",
    "CuLa1.894O4Sr0.106",
    "Ba1.91Cu3Na0.09O7Y",
    "Ba1.25Ca0.48Cu3La0.98O6.94Pr0.23",
    "Ba1.24Ca0.42Cu3La1.04O7.1Pr0.18",
    "Ba1.85Cu3La0.15O7Y",
    "As3CaCr0.84Fe3.16",
    "CuEu0.21O1.94Sr0.79",
    "C2Bi2Cu3O16Sr5",
    "Fe0.2Ti0.8",
    "Ce0.7Cu2Gd1.3O9.9RuSr2",
    "Ca0.5Cu2Hg0.4O7Pr1.1Sr2",
    "Cu2NdO7Sr2Tl",
    "Ba2Ca1.95Cu3.25O9.952Tl1.66Y0.05",
    "Ce0.5Cu2.5Mn0.5O9.15Sr2Y1.5",
    "Ba2Cu2.78O7Y",
    "Ba2Cu1.06O6Tl1.94",
    "Bi2Ca1.7Cu3O10Pb0.3Sr2",
    "Ba1.6Cu3O6.817Sr0.4Yb",
    "Ba1.5Ca0.5Cu3LaO6.54",
    "Ba2Ca0.8Cu2Nd0.2O6.86Tl0.96",
    "Fe0.5V1.5Zr",
    "Ba2Ca0.27Co0.3Cu2.7O7Y0.73",
    "CuLa1.83Na0.16O4",
    "Bi2Ca1.7Cu3F4O8Pb0.3Sr2",
    "Ba2Cu2.8Ni0.2O6.85Y",
    "Ba2Cu2.93O6.91Y",
    "Fe0.4Mo6S8Sn",
    "Ba2Cu2.94Li0.06O6.91Y",
    "Ba2CuHg0.88O4.87S0.18",
    "Ca0.65Cu2La1.6O6Pr0.35Sr0.4",
    "Ba2Cu2.943Fe0.019O6.132Pr0.004Y0.997",
    "Cu1.47Mo3S4",
    "Ba1.28Ca0.4Cu3La1.11O7.14Y0.2",
    "Bi2.1Ca1.12Cu2O8Sr1.78",
    "Ba2Cu2.82Li0.18O6.77Y0.98",
    "Ba2Co0.38Cu2.62O6.91Y",
    "Ca0.36Cu2.17Lu0.97O7Pb0.83Sr1.67",
    "Ba2Cu2.74Fe0.22O7.24Sm",
    "Ba2CuHg0.752Mo0.252O4.584",
    "Ce0.2CuNd1.8O4",
    "Ba2Cu2.764Fe0.236O7.36Sm",
    "Ba2Cu3O6.26Sm",
    "Ba2Cu2.98O6.92Y",
    "Ba1.5Cu3La0.5O7.22Y0.92",
    "La₂PrNi₂O₇",
    "Pr₄Ni₃O₁₀",
    "(InSe₂)₀.₁₂NbSe₂",
    "Sc",
    "LaH₁₀",
    "YH₉",
    "YH₁₀",
    "ThH₁₀",
    "CaH₆",
    "HgBa₂Ca₂Cu₃O₈",
    "Tl₂Ba₂Ca₂Cu₃O₁₀",
    "Bi₂Sr₂Ca₂Cu₃O₁₀",
    "Tl₂Ba₂CaCu₂O₈",
    "Bi₂Sr₂CaCu₂O₈",
    "YBa₂Cu₃O₇",
    "HgBa₂CaCu₂O₆",
    "HgBa₂CuO₄",
    "Tl₂Ba₂CuO₆",
    "MgB₂",
    "BaFe₂As₂",
    "LaFeAsO",
    "SmFeAsO",
    "FeSe",
    "LiFeAs",
    "NaFeAs",
    "Nb₃Sn",
    "NbTi",
    "V₃Si",
    "CeCu₂Si₂",
    "UPt₃",
    "UBe₁₃",
    "URu₂Si₂",
    "CeCoIn₅",
    "CeRhIn₅",
    "K₃C₆₀",
    "Cs₃C₆₀",
    "NbSe₂",
    "MoS₂",
    "WS₂",
    "Fe₃Sn₂",
    "KV₃Sb₅",
    "RbV₃Sb₅",
"CsV₃Sb₅",
"ScV₆Sn₆",
"KV₃Bi₅",
"CsV₃Bi₅",
"Co₃Sn₂S₂",
"Mn₃Sn",
"Mn₃Ge",
"ZrSiS",
"Cd₃As₂",
"Na₃Bi",
"FeGe",

"SrAuH3",
"SrZnH3",
"YSc2H24",
"CaBC",
"MgIrH",
"Ti0.2Nb0.2Ta0.2Mo0.2W0.2C0.7N0.3",
"La3Ni2O7",
"(TaNb)0.67(HfZrTi)0.33",

#new ones
"CsTi3Bi5",
"RbTi3Bi5",
"KTi3Bi5",
"Ti6Bi6",
"CsV3Bi5",
"KV3Bi5",
"RbV3Bi5",
"ScV6Sn6",
"TbV6Sn6",
"Na3Bi",
"ZrSiSe",
"ZrGeTe",
"HfTe5",
"TaIrTe4",
"EuCd2As2",
"MnBi2Te4",
"CrAs",
"UTe2"
]

print(f"Total compositions to predict: {len(new_compositions)}")
print(f"Using {len(all_models)} models in ensemble\n")

results = []
failed_predictions = []

for idx, comp in enumerate(new_compositions, 1):
    try:
        mean_tc, std_tc = predict_tc_ensemble(
            comp, all_models, all_scalers, element_features_dict, debug=False
        )
        
        print(f"{idx}/{len(new_compositions)}: {comp:<40} -> Tc = {mean_tc:7.2f} ± {std_tc:5.2f} K")
        
        results.append({
            'Composition': comp,
            'Predicted_Tc_Mean': mean_tc,
            'Predicted_Tc_Std': std_tc,
            'Predicted_Tc_Min': mean_tc - std_tc,
            'Predicted_Tc_Max': mean_tc + std_tc,
            'CI_95_Lower': mean_tc - 1.96 * std_tc,
            'CI_95_Upper': mean_tc + 1.96 * std_tc
        })
        
    except Exception as e:
        print(f"{idx}/{len(new_compositions)}: {comp:<40} -> ERROR: {str(e)}")
        failed_predictions.append({'Composition': comp, 'Error': str(e)})
        results.append({
            'Composition': comp,
            'Predicted_Tc_Mean': None,
            'Predicted_Tc_Std': None,
            'Predicted_Tc_Min': None,
            'Predicted_Tc_Max': None,
            'CI_95_Lower': None,
            'CI_95_Upper': None
        })


print("SAVING RESULTS")

df_results = pd.DataFrame(results)
df_results.to_csv("ensemble_predictions.csv", index=False)
print(f" Ensemble predictions saved to: ensemble_predictions.csv")
print(f"  Total predictions: {len(results)}")
print(f"  Successful: {len(results) - len(failed_predictions)}")
print(f"  Failed: {len(failed_predictions)}")



if failed_predictions:
    df_failed = pd.DataFrame(failed_predictions)
    df_failed.to_csv("failed_predictions.csv", index=False)
    print(f"Failed predictions saved to: failed_predictions.csv")



print("PREDICTION SUMMARY STATISTICS")

df_successful = df_results[df_results['Predicted_Tc_Mean'].notna()]

if len(df_successful) > 0:
    print(f"Predicted Tc Statistics:")
    print(f"  Mean:   {df_successful['Predicted_Tc_Mean'].mean():.2f} K")
    print(f"  Median: {df_successful['Predicted_Tc_Mean'].median():.2f} K")
    print(f"  Std:    {df_successful['Predicted_Tc_Mean'].std():.2f} K")
    print(f"  Min:    {df_successful['Predicted_Tc_Mean'].min():.2f} K")
    print(f"  Max:    {df_successful['Predicted_Tc_Mean'].max():.2f} K")
    
    print(f"\nAverage Prediction Uncertainty:")
    print(f"  Mean Std: {df_successful['Predicted_Tc_Std'].mean():.2f} K")
    print(f"  Median Std: {df_successful['Predicted_Tc_Std'].median():.2f} K")
    
  
    print("TOP 10 PREDICTED SUPERCONDUCTORS (by Tc)")

    top_10 = df_successful.nlargest(10, 'Predicted_Tc_Mean')
    for i, row in enumerate(top_10.itertuples(), 1):
        print(f"{i:2d}. {row.Composition:<40} {row.Predicted_Tc_Mean:7.2f} ± {row.Predicted_Tc_Std:5.2f} K")
    
    # Materials with low uncertainty
    print("TOP 10 MOST CONFIDENT PREDICTIONS (lowest uncertainty)")

    low_uncertainty = df_successful.nsmallest(10, 'Predicted_Tc_Std')
    for i, row in enumerate(low_uncertainty.itertuples(), 1):
        print(f"{i:2d}. {row.Composition:<40} {row.Predicted_Tc_Mean:7.2f} ± {row.Predicted_Tc_Std:5.2f} K")
    
    # Plot distribution
    print("GENERATING VISUALIZATION")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Histogram of predicted Tc
    axes[0, 0].hist(df_successful['Predicted_Tc_Mean'], bins=50, 
                    color='steelblue', alpha=0.7, edgecolor='black')
    axes[0, 0].set_xlabel('Predicted Tc (K)', fontsize=12)
    axes[0, 0].set_ylabel('Count', fontsize=12)
    axes[0, 0].set_title('Distribution of Predicted Tc', fontsize=14, fontweight='bold')
    axes[0, 0].grid(alpha=0.3)
    
    # Plot 2: Histogram of uncertainties
    axes[0, 1].hist(df_successful['Predicted_Tc_Std'], bins=50, 
                    color='coral', alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Prediction Uncertainty (Std, K)', fontsize=12)
    axes[0, 1].set_ylabel('Count', fontsize=12)
    axes[0, 1].set_title('Distribution of Prediction Uncertainties', fontsize=14, fontweight='bold')
    axes[0, 1].grid(alpha=0.3)
    
    # Plot 3: Scatter plot - Tc vs Uncertainty
    axes[1, 0].scatter(df_successful['Predicted_Tc_Mean'], 
                       df_successful['Predicted_Tc_Std'],
                       alpha=0.5, s=30, c='green', edgecolors='black', linewidth=0.5)
    axes[1, 0].set_xlabel('Predicted Tc (K)', fontsize=12)
    axes[1, 0].set_ylabel('Prediction Uncertainty (K)', fontsize=12)
    axes[1, 0].set_title('Predicted Tc vs Uncertainty', fontsize=14, fontweight='bold')
    axes[1, 0].grid(alpha=0.3)
    
    # Plot 4: Top 20 compounds
    top_20 = df_successful.nlargest(20, 'Predicted_Tc_Mean')
    y_pos = np.arange(len(top_20))
    axes[1, 1].barh(y_pos, top_20['Predicted_Tc_Mean'], 
                    xerr=top_20['Predicted_Tc_Std'],
                    alpha=0.7, color='purple', capsize=3)
    axes[1, 1].set_yticks(y_pos)
    axes[1, 1].set_yticklabels(top_20['Composition'], fontsize=8)
    axes[1, 1].set_xlabel('Predicted Tc (K)', fontsize=12)
    axes[1, 1].set_title('Top 20 Predicted Superconductors', fontsize=14, fontweight='bold')
    axes[1, 1].grid(alpha=0.3, axis='x')
    axes[1, 1].invert_yaxis()
    
    plt.tight_layout()
    plt.savefig('ensemble_predictions_summary.png', dpi=300, bbox_inches='tight')
    print("Visualization saved to: ensemble_predictions_summary.png")
    plt.show()


print("ENSEMBLE PREDICTION COMPLETE!")


## 4. SHAP Interpretability Analysis

Perform SHAP analysis on the top 5 models to understand feature importance:

In [ ]:
print("SHAP ANALYSIS - TOP 5 MODELS WITH NUMERICAL STABILITY")

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Feature names (make sure this is defined)
feature_names = [
    'atomic_number', 'atomic_volume', 'block_encoded', 'density', 'dipole_polarizability',
    'electron_affinity', 'evaporation_heat', 'fusion_heat', 'group_id', 'lattice_constant',
    'lattice_structure', 'melting_point', 'period', 'specific_heat', 'thermal_conductivity',
    'vdw_radius', 'covalent_radius_pyykko', 'en_pauling', 'atomic_weight', 'atomic_radius_rahm',
    'first_ionization_energy', 'valence_electrons', 'stoichiometry'
]

# ========================================================================
# STEP 1: SELECT TOP 5 MODELS BASED ON TEST R²
# ========================================================================
print("\n" + "="*80)
print("SELECTING TOP 5 MODELS")
print("="*80)

# Load metrics to find best models
metrics_df = pd.read_csv('saved_models/all_runs_metrics.csv')
print(f"\nAll 50 models R² range: {metrics_df['test_r2'].min():.4f} to {metrics_df['test_r2'].max():.4f}")
print(f"Mean R²: {metrics_df['test_r2'].mean():.4f} ± {metrics_df['test_r2'].std():.4f}")

# Get indices of top 5 models
top_5_indices = metrics_df.nlargest(5, 'test_r2')['run'].values - 1  # -1 because run numbers start at 1
print(f"\nTop 5 model indices: {top_5_indices}")
print("\nTop 5 models performance:")
for idx in top_5_indices:
    r2 = metrics_df.iloc[idx]['test_r2']
    mae = metrics_df.iloc[idx]['test_mae']
    rmse = metrics_df.iloc[idx]['test_rmse']
    print(f"  Model {idx+1}: R²={r2:.4f}, MAE={mae:.2f}K, RMSE={rmse:.2f}K")

# Select top 5 models and scalers
top_5_models = [all_models[idx] for idx in top_5_indices]
top_5_scalers = [all_scalers[idx] for idx in top_5_indices]

# ========================================================================
# STEP 2: PREPARE DATA FOR SHAP ANALYSIS
# ========================================================================
print("\n" + "="*80)
print("PREPARING DATA FOR SHAP ANALYSIS")
print("="*80)

# Use fewer samples for numerical stability
X_common_test = X_deepset[:1000]
y_common_test = y[:1000]

num_background_samples = 50   # Reduced for stability
num_explain_samples = 100      # Reduced for stability

print(f"Using {len(X_common_test)} total samples")
print(f"Background samples: {num_background_samples}")
print(f"Samples to explain: {num_explain_samples}")

# ========================================================================
# STEP 3: COMPUTE SHAP VALUES WITH NUMERICAL STABILITY
# ========================================================================
print("\n" + "="*80)
print("COMPUTING SHAP VALUES (TOP 5 MODELS)")
print("="*80)

all_shap_values = []

for model_num, (model_idx, model, scaler) in enumerate(zip(top_5_indices, top_5_models, top_5_scalers)):
    print(f"\nModel {model_num + 1}/5 (Run {model_idx + 1}): Computing SHAP values...")
    
    # Normalize the test data with this model's scaler
    X_test_normalized = scaler.transform(X_common_test.reshape(-1, 23)).reshape(-1, max_elements, 23)
    
    # Create background dataset
    background_data = X_test_normalized[:num_background_samples]
    
    # Data to explain
    data_to_explain = X_test_normalized[num_background_samples:num_background_samples+num_explain_samples]
    
    # ========================================================================
    # KEY FIX: Use element-wise SHAP instead of flattened
    # ========================================================================
    # Instead of flattening to 230D, we'll compute SHAP per element position
    # and aggregate, which is more numerically stable
    
    def model_predict(x):
        """Wrapper function for SHAP that handles reshaping"""
        if len(x.shape) == 2:
            # x is (batch_size, 230) - reshape to (batch_size, 10, 23)
            x = x.reshape(-1, max_elements, 23)
        return model.predict(x, verbose=0)
    
    # Flatten for KernelExplainer
    background_flat = background_data.reshape(num_background_samples, -1)
    explain_flat = data_to_explain.reshape(num_explain_samples, -1)
    
    try:
        # Use fewer samples for KernelExplainer to improve stability
        explainer = shap.KernelExplainer(model_predict, background_flat)
        
        np.random.seed(42 + model_num)  # Different seed per model
        
        # Compute SHAP values with reduced samples
        shap_values = explainer.shap_values(explain_flat, nsamples=50)  # Further reduced
        
        # Handle list output
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        # ========================================================================
        # CRITICAL: CLIP EXTREME VALUES TO PREVENT NUMERICAL EXPLOSIONS
        # ========================================================================
        # Sometimes SHAP can produce extremely large values due to numerical issues
        # We'll clip at 99.9th percentile to remove outliers
        shap_abs = np.abs(shap_values)
        threshold = np.percentile(shap_abs, 99.9)
        shap_values_clipped = np.clip(shap_values, -threshold, threshold)
        
        print(f"  Raw SHAP range: [{shap_values.min():.2e}, {shap_values.max():.2e}]")
        print(f"  Clipped at: ±{threshold:.2e}")
        print(f"  After clipping: [{shap_values_clipped.min():.2e}, {shap_values_clipped.max():.2e}]")
        
        # Reshape back to (num_samples, max_elements, 23)
        shap_values_reshaped = shap_values_clipped.reshape(num_explain_samples, max_elements, 23)
        all_shap_values.append(shap_values_reshaped)
        
        print(f"  ✓ Completed model {model_num + 1}")
        
    except Exception as e:
        print(f"  ✗ Error with model {model_num + 1}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n✓ SHAP values computed for {len(all_shap_values)} models!")

if len(all_shap_values) == 0:
    print("\n❌ ERROR: No SHAP values were successfully computed!")
    print("This suggests a fundamental issue with the model or data.")
    exit(1)

# ========================================================================
# STEP 4: AGGREGATE SHAP VALUES USING ABSOLUTE VALUES
# ========================================================================
print("\n" + "="*80)
print("AGGREGATING SHAP VALUES")
print("="*80)

# Convert to numpy array
all_shap_values = np.array(all_shap_values)  # Shape: (n_models, num_samples, max_elements, 23)
print(f"All SHAP values shape: {all_shap_values.shape}")

# Take absolute value BEFORE averaging
all_shap_abs = np.abs(all_shap_values)

# Average across models first, then across samples and elements
ensemble_shap_abs = np.mean(all_shap_abs, axis=0)  # Shape: (num_samples, max_elements, 23)
print(f"Ensemble absolute SHAP shape: {ensemble_shap_abs.shape}")

# Flatten across samples and elements to get feature importance
ensemble_shap_abs_2d = ensemble_shap_abs.reshape(-1, 23)

# Calculate global feature importance
mean_abs_shap = np.mean(ensemble_shap_abs_2d, axis=0)
median_abs_shap = np.median(ensemble_shap_abs_2d, axis=0)  # Median is more robust to outliers
shap_importance_std = np.std(ensemble_shap_abs_2d, axis=0)

print(f"Feature importance shape: {mean_abs_shap.shape}")
print(f"Mean feature importance range: [{mean_abs_shap.min():.6f}, {mean_abs_shap.max():.6f}]")
print(f"Median feature importance range: [{median_abs_shap.min():.6f}, {median_abs_shap.max():.6f}]")

# ========================================================================
# STEP 5: DISPLAY AND SAVE RESULTS
# ========================================================================
print("\n" + "="*80)
print("GLOBAL FEATURE IMPORTANCE (Top 5 Models)")
print("="*80)
print(f"\n{'Rank':<6} {'Feature':<30} {'Mean |SHAP|':<14} {'Median |SHAP|':<14} {'Std Dev':<12}")
print("-" * 80)

# Sort by MEDIAN (more robust to outliers)
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Mean_Abs_SHAP': mean_abs_shap,
    'Median_Abs_SHAP': median_abs_shap,
    'Std_Dev': shap_importance_std,
})
importance_df['Rank'] = importance_df['Median_Abs_SHAP'].rank(ascending=False)
importance_df = importance_df.sort_values('Median_Abs_SHAP', ascending=False)

for idx, row in importance_df.iterrows():
    print(f"{int(row['Rank']):<6} {row['Feature']:<30} {row['Mean_Abs_SHAP']:<14.6f} "
          f"{row['Median_Abs_SHAP']:<14.6f} {row['Std_Dev']:<12.6f}")

# Save results
importance_df.to_csv('saved_models/shap_importance_top5_models.csv', index=False)
np.save('saved_models/shap_values_top5_models.npy', ensemble_shap_abs)

print("\n✓ Results saved to:")
print("  - saved_models/shap_importance_top5_models.csv")
print("  - saved_models/shap_values_top5_models.npy")

# ========================================================================
# STEP 6: VISUALIZATIONS
# ========================================================================
print("\n" + "="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)

# 1. Bar plot with median importance (more robust)
print("\n1. Creating feature importance bar plot...")
fig, ax = plt.subplots(figsize=(12, 8))

sorted_importance = importance_df.sort_values('Median_Abs_SHAP', ascending=True)
y_pos = np.arange(len(sorted_importance))

ax.barh(y_pos, sorted_importance['Median_Abs_SHAP'], 
        alpha=0.8, color='steelblue', label='Median |SHAP|')
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_importance['Feature'], fontsize=10)
ax.set_xlabel('Median |SHAP Value| (Top 5 Models)', fontsize=12)
ax.set_title('Feature Importance Based on SHAP\n(Using Top 5 Best Performing Models)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
ax.legend()

plt.tight_layout()
plt.savefig('saved_models/shap_importance_top5_bars.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Top 10 features
print("\n2. Creating top 10 features plot...")
fig, ax = plt.subplots(figsize=(10, 6))

top_10 = importance_df.head(10).sort_values('Median_Abs_SHAP', ascending=True)
y_pos = np.arange(len(top_10))

colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_10)))
bars = ax.barh(y_pos, top_10['Median_Abs_SHAP'], color=colors, alpha=0.8)

# Add value labels
for i, (bar, v) in enumerate(zip(bars, top_10['Median_Abs_SHAP'])):
    ax.text(v, i, f' {v:.4f}', va='center', fontsize=9)

ax.set_yticks(y_pos)
ax.set_yticklabels(top_10['Feature'], fontsize=11, fontweight='bold')
ax.set_xlabel('Median |SHAP Value|', fontsize=12)
ax.set_title('Top 10 Most Important Features\n(Top 5 Models)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('saved_models/shap_top10_features.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Comparison: Mean vs Median importance
print("\n3. Creating mean vs median comparison...")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Sort by median for consistent ordering
sorted_df = importance_df.sort_values('Median_Abs_SHAP', ascending=True)
y_pos = np.arange(len(sorted_df))

# Mean importance
ax1.barh(y_pos, sorted_df['Mean_Abs_SHAP'], alpha=0.7, color='coral')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(sorted_df['Feature'], fontsize=9)
ax1.set_xlabel('Mean |SHAP Value|', fontsize=12)
ax1.set_title('Feature Importance - Mean', fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Median importance  
ax2.barh(y_pos, sorted_df['Median_Abs_SHAP'], alpha=0.7, color='steelblue')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(sorted_df['Feature'], fontsize=9)
ax2.set_xlabel('Median |SHAP Value|', fontsize=12)
ax2.set_title('Feature Importance - Median (More Robust)', fontsize=13, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('saved_models/shap_mean_vs_median.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Distribution of SHAP values per feature (boxplot)
print("\n4. Creating SHAP distribution boxplot...")
fig, ax = plt.subplots(figsize=(14, 6))

# Prepare data for boxplot
shap_data_by_feature = [ensemble_shap_abs_2d[:, i] for i in range(23)]

bp = ax.boxplot(shap_data_by_feature, positions=range(23), widths=0.6,
                patch_artist=True, showfliers=False)

for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

ax.set_xticks(range(23))
ax.set_xticklabels(feature_names, rotation=45, ha='right', fontsize=9)
ax.set_ylabel('|SHAP Value| Distribution', fontsize=12)
ax.set_title('Distribution of Absolute SHAP Values Per Feature\n(Across all samples)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('saved_models/shap_distribution_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("SHAP ANALYSIS COMPLETE!")
print("="*80)
print("\nKey Findings:")
print(f"✓ Most Important Feature:  {importance_df.iloc[0]['Feature']} "
      f"(Median SHAP = {importance_df.iloc[0]['Median_Abs_SHAP']:.6f})")
print(f"✓ 2nd Most Important:      {importance_df.iloc[1]['Feature']} "
      f"(Median SHAP = {importance_df.iloc[1]['Median_Abs_SHAP']:.6f})")
print(f"✓ 3rd Most Important:      {importance_df.iloc[2]['Feature']} "
      f"(Median SHAP = {importance_df.iloc[2]['Median_Abs_SHAP']:.6f})")
print(f"\n✓ Least Important Feature: {importance_df.iloc[-1]['Feature']} "
      f"(Median SHAP = {importance_df.iloc[-1]['Median_Abs_SHAP']:.6f})")

print(f"\n✓ Analysis based on top 5 best models (out of 50)")
print(f"✓ Used median aggregation for robustness to outliers")
print(f"✓ Applied numerical clipping to prevent SHAP explosions")
print(f"✓ All plots saved to saved_models/ directory")